In [1]:
import pandas as pd
import numpy as np

import sidetable as stb

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew
from scipy.stats import kurtosistest
from scipy import stats
from sklearn.preprocessing import OneHotEncoder


import pickle
import listas as lst

from datetime import datetime

In [2]:
df = pd.read_pickle("../data/bikes_predicciones.pkl")
df.sample()

,estacion,año,mes,festivo,dia_sem,laborable,prevision_tiempo,sensacion_termica,humedad,velocidad_viento
172,Verano,2018,junio,0,viernes,1,1,34.69145,70.75,11.541554


In [3]:
with open ("../data/robust_total.pkl", "rb") as file:
    robust_total = pickle.load(file) 

In [4]:
with open ("../data/dt_total_pred.pkl", "rb") as file:
    dt_total_pred = pickle.load(file)

In [5]:
# creamos un diccionario con datos ficticios:

dicc_usuario_total = {'estacion': 'Verano', 'año': '2019', 'mes': 'septiembre', 'festivo': 0, 'dia_sem': 'domingo', 'laborable': 0,
                'prevision_tiempo': 1, 'sensacion_termica': 30.52440, 'humedad': 64.6667, 'velocidad_viento': 19.000061}

In [6]:
# creamos df

df_usuario_total = pd.DataFrame(dicc_usuario_total, index= [0])
df_cat_total = df_usuario_total.drop(columns=["laborable", "dia_sem"], axis = 1)

In [7]:
for dic, col in zip(lst.lista_dict_total, df_cat_total.columns):   
    
    df_usuario_total[col] = df_usuario_total[col].map(dic)

In [8]:
df_usuario_total

,estacion,año,mes,festivo,dia_sem,laborable,prevision_tiempo,sensacion_termica,humedad,velocidad_viento
0,2.5,1.5,2.6,NaN,domingo,0,2.0,30.5244,64.6667,19.000061


In [9]:
def one_hot_encoder_one(df,columna,keep_first=True):
    
    oh = OneHotEncoder()

    transformados = oh.fit_transform(df[[columna]])
 
    oh_df = pd.DataFrame(transformados.toarray())

    oh_df.columns = oh.get_feature_names_out()
    return oh_df

In [10]:
df_one_hot = one_hot_encoder_one(df_usuario_total, "dia_sem")

In [11]:
df_one_hot

,dia_sem_domingo
0,1.0


In [12]:
df_usuario_total= df_usuario_total.drop(columns=["dia_sem"], axis= 1)

In [13]:
df_usuario_total = pd.concat([df_usuario_total, df_one_hot], axis= 1)

In [14]:
df_usuario_total

,estacion,año,mes,festivo,laborable,prevision_tiempo,sensacion_termica,humedad,velocidad_viento,dia_sem_domingo
0,2.5,1.5,2.6,NaN,0,2.0,30.5244,64.6667,19.000061,1.0


In [15]:
reemplazo = {"estacion": "estacion_map", "año": "año_map", "mes":"mes_map", "dia_sem_domingo": "dia_sem_domingo", "laborable":"laborable_map", "prevision_tiempo":"prevision_tiempo_map", "festivo": "festivo"}

In [16]:
df_usuario_total.rename(columns=reemplazo, inplace=True)

In [17]:
df_usuario_total["festivo"] = df["festivo"]

In [18]:
df_usuario_total = df_usuario_total.reindex(columns=['sensacion_termica', 'humedad', 'velocidad_viento', 'estacion_map',
       'año_map', 'mes_map', 'dia_sem_domingo', 'laborable_map',
       'prevision_tiempo_map', 'festivo'])

In [19]:
dicc_ceros:{"dia_sem_jueves":0,	"dia_sem_lunes":0,"dia_sem_martes":0, "dia_sem_miercoles":0,"dia_sem_sabado": 0,"dia_sem_viernes":0}

In [20]:
dicc_ceros

NameError: name 'dicc_ceros' is not defined

In [ ]:
df_ceros = pd.DataFrame(dicc_ceros, inplace= True)
df_ceros

NameError: name 'dicc_ceros' is not defined

In [ ]:
df_num = df_usuario_total[["sensacion_termica", "humedad", "velocidad_viento"]]

In [ ]:
df_num

,sensacion_termica,humedad,velocidad_viento
0,30.5244,64.6667,19.000061


In [ ]:
robust_total.transform(df_num)

array([[0.45412707, 0.09651821, 1.04416759]])

In [ ]:
dt_total_pred.predict(df_usuario_total)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- festivo
- laborable_map
Feature names seen at fit time, yet now missing:
- dia_sem_jueves
- dia_sem_lunes
- dia_sem_martes
- dia_sem_miercoles
- dia_sem_sabado
- ...


Comprobado con el dataFrame original, se alcanzaron 2512 alquileres casuales. Nuestra predicción calcula 2201 alquiler.
Hay una diferencia de 311 bicis. El modelo tenía un margen de error de 303

In [ ]:
df = pd.read_pickle("../data/outliers_gestion.pkl")

In [ ]:
df[df["sensacion_termica"] == 30.52440]

,fecha,estacion,año,mes,festivo,dia_sem,laborable,prevision_tiempo,sensacion_termica,humedad,velocidad_viento,alquiler_casual,alquiler_registrado,total_alquileres
629,2019-09-22,Verano,2019,septiembre,0,domingo,0,1,30.5244,64.6667,19.000061,2512,5883,8395
